In [22]:
import cv2
import numpy as np
import queue
from keras.models import load_model

MODEL_SETTING_DICT = {'violenceModel_baseOn_inceptionV3.h5':{'imgsz':224,
                                                             'normalize':False,
                                                             'mean_subtraction':True
                                                            },
                      'violenceModel_baseOn_MobileNetV2.h5':{'imgsz':128,
                                                             'normalize':True,
                                                             'mean_subtraction':True
                                                            }
                     }
MEAN = np.array([123.68, 116.779, 103.939][::1], dtype="float32")


class ViolencePredictor(object):
    '''
    model_name: ['violenceModel_baseOn_inceptionV3.h5','violenceModel_baseOn_MobileNetV2.h5']
    '''
    def __init__(self,
                 model_name: str = "violenceModel_baseOn_inceptionV3.h5",
                 #imgsz: int = 128,
                 queue_num: int = 30,
                 conf_thres: float = 0.25,
                 num_thres: int = 20) -> None:
        self.model_name = model_name
        self.model = load_model(f"models/{model_name}")
        self.results = queue.Queue()
        self.imgsz = MODEL_SETTING_DICT[model_name]['imgsz']
        self.queue_num = queue_num
        self.conf_thres = conf_thres
        self.num_thres = num_thres

    def process(self, img):
        # pre-process image
        img = self._preprocess(img)
        # run infer
        preds = self.model.predict(np.expand_dims(img, axis=0))[0]
        print(preds)
        # count result
        if preds > self.conf_thres:
            self.results.put(True)
        else:
            self.results.put(False)
        if self.results.qsize() > self.queue_num:
            self.results.get()
        # judge
        if sum(self.results.queue) > self.num_thres:
            return True
        else:
            return False
        
    def _preprocess(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.imgsz, self.imgsz)).astype("float32")
        img = img.reshape(self.imgsz, self.imgsz, 3)
        img = img / 255 if MODEL_SETTING_DICT[self.model_name]['normalize'] else img
        img = img - MEAN if MODEL_SETTING_DICT[self.model_name]['mean_subtraction'] else img
        return img
#     def _save_infer_image

In [ ]:

i = 0
cap = cv2.VideoCapture("dataset/Violence/V_129.mp4")
model_name_list = ['violenceModel_baseOn_inceptionV3.h5','violenceModel_baseOn_MobileNetV2.h5']
vp = ViolencePredictor(model_name = model_name_list[0])
# cap = cv2.VideoCapture("datasets/videos/NonViolence/NV_129.mp4")
while i < 90:
    i += 1
    _, img = cap.read()
    violence = vp.process(img)
    print(violence)

In [9]:
model_name_list[1]

'violenceModel_baseOn_MobileNetV2.h5'

In [11]:
cap.read()

(False, None)

In [15]:
load_model(f"models/{model_name_list[1]}")

TypeError: ('Keyword argument not understood:', 'keepdims')

In [21]:
import tensorflow as tf; print(tf.__version__)

2.4.1
